<a href="https://colab.research.google.com/github/eshagit/transformers_BERT/blob/master/BERT_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:

!pip install bertviz
from transformers import BertModel,BertConfig,BertTokenizer
import torch
from bertviz import model_view, head_view
import pandas as pd



In [5]:
model = BertModel.from_pretrained('bert-base-uncased')# Bert model contains 12 encoding layes thats why len is 12

print(len(model.encoder.layer))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

12


In [6]:
print(model.encoder.layer[0])  # Each layer has attention layer which has query key value

BertLayer(
  (attention): BertAttention(
    (self): BertSdpaSelfAttention(
      (query): Linear(in_features=768, out_features=768, bias=True)
      (key): Linear(in_features=768, out_features=768, bias=True)
      (value): Linear(in_features=768, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (output): BertSelfOutput(
      (dense): Linear(in_features=768, out_features=768, bias=True)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (intermediate): BertIntermediate(
    (dense): Linear(in_features=768, out_features=3072, bias=True)
    (intermediate_act_fn): GELUActivation()
  )
  (output): BertOutput(
    (dense): Linear(in_features=3072, out_features=768, bias=True)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
)


In [7]:
config = BertConfig()  #BertModel conatins 12 encoders and each encoders has 12 attendtion heads ie 12 hidden small attendtion layers
print(config)

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.46.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
text = "My frined told me about this class and i love it so far,she was right"
tokens = tokenizer.encode(text)
inputs = torch.tensor(tokens).unsqueeze(0)
print(inputs)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tensor([[  101,  2026, 10424, 21280,  2409,  2033,  2055,  2023,  2465,  1998,
          1045,  2293,  2009,  2061,  2521,  1010,  2016,  2001,  2157,   102]])


In [9]:
attention = model(inputs,output_attentions=True)[2] # model contains 12 encoders we are getting encoder of index 2 and each encoder contains again 12 attention

final_attention = attention[-1].mean(1)[0]  # get last attention and average it (mean)

attention_df = pd.DataFrame(final_attention.detach()).applymap(float).round(3)

attention_df.columns = tokenizer.convert_ids_to_tokens(tokens)
attention_df.index = tokenizer.convert_ids_to_tokens(tokens)
print(attention_df)

BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


        [CLS]     my     fr  ##ined   told     me  about   this  class    and  \
[CLS]   0.067  0.034  0.035   0.068  0.032  0.013  0.037  0.036  0.083  0.042   
my      0.032  0.056  0.030   0.043  0.019  0.013  0.018  0.011  0.010  0.017   
fr      0.029  0.019  0.102   0.088  0.018  0.007  0.007  0.005  0.008  0.009   
##ined  0.033  0.025  0.056   0.130  0.021  0.011  0.006  0.005  0.009  0.014   
told    0.022  0.018  0.031   0.046  0.088  0.011  0.024  0.010  0.013  0.015   
me      0.030  0.024  0.023   0.043  0.026  0.020  0.019  0.014  0.011  0.024   
about   0.033  0.017  0.014   0.021  0.034  0.015  0.112  0.022  0.016  0.017   
this    0.038  0.017  0.012   0.016  0.012  0.015  0.022  0.067  0.020  0.015   
class   0.035  0.009  0.010   0.015  0.012  0.008  0.018  0.027  0.119  0.011   
and     0.037  0.019  0.017   0.034  0.015  0.012  0.009  0.009  0.009  0.083   
i       0.025  0.020  0.024   0.042  0.019  0.010  0.012  0.011  0.010  0.020   
love    0.018  0.007  0.009 

<ipython-input-9-2c5cfc97baa8>:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  attention_df = pd.DataFrame(final_attention.detach()).applymap(float).round(3)


In [10]:
tokens_as_list = tokenizer.convert_ids_to_tokens(inputs[0])
head_view(attention,tokens_as_list)

<IPython.core.display.Javascript object>